In [1]:
def whatRound(round, elimination, maxRounds):
    # if single elimination
    if elimination == "s":
        if maxRounds == 4:
            roundName = {
                4 : "Finals",
                3 : "Semi-Finals",
                2 : "Quarter Finals",
                1 : "Ro16"
            }
            return roundName[round]
        elif maxRounds == 5:
            roundName = {
                5 : "Finals",
                4 : "Semi-Finals",
                3 : "Quarter Finals",
                2 : "Ro16",
                1 : "Ro32"
            }
            return roundName[round]
        elif maxRounds == 6:
            roundName = {
                6 : "Finals",
                5 : "Semi-Finals",
                4 : "Quarter Finals",
                3 : "Ro16",
                2 : "Ro32",
                1 : "Ro64"
            }
            return roundName[round]
        else:
            print(f"There is no name for a tournament with a max round of {maxRounds}")
            return
    # if double elimination
    elif elimination == "d":
        if maxRounds == 4:
            roundName = {
                4 : "Finals",
                3 : "Winners Final",
                2 : "Winners Semi-Finals",
                1 : "Winners Quarter Finals",
                -1 : "Losers Round 1",
                -2 : "Losers Quarter Finals",
                -3 : "Losers Semi-Finals",
                -4 : "Losers Finals"
            }
            return roundName[round]
        elif maxRounds == 5:
            roundName = {
                5 : "Finals",
                4 : "Winners Final",
                3 : "Winners Semi-Finals",
                2 : "Winners Quarter Finals",
                1 : "Winners Ro16",
                -1 : "Losers Round 1",
                -2 : "Losers Round 2",
                -3 : "Losers Quarter Finals",
                -4 : "Losers Semi-Finals",
                -5 : "Losers Final"
            }
            return roundName[round]
        else:
            print(f"There is no name for a tournament with a max round of {maxRounds}")
            return
    else:
        print("No elimination type given")
        return


In [2]:
import sqlite3
def numberOfMMC(NAME):
    conn: sqlite3.Connection = sqlite3.connect("mmc.db")
    conn.execute("PRAGMA foreign_keys = 1")
    c: sqlite3.Cursor = conn.cursor()

    cnames = [] # for collecting all the challonge names a player has had
    specificMMC = [] # for recording the specific editions a player has played in
    c.execute("SELECT * FROM ChallongeNames WHERE NAME = ?", (NAME,)) # query all challonge names a player has had
    results = c.fetchall()
    for r in results:
        cnames.append(r[0])
    for cn in cnames:
        c.execute("SELECT * FROM Participants WHERE CNAME = ?",(cn,))
        results = c.fetchall()
        for r in results:
            c.execute("SELECT NUMBER FROM MMC WHERE TOURNAMENTID = ?", (r[3],))
            results2 = c.fetchone()
            if results2[0]:
                specificMMC.append(results2[0])
    specificMMC.sort()
    # print results
    if len(specificMMC) == 1:
        print(f"{NAME} has played in {len(specificMMC)} MMC")
        print(f"The specific edition {NAME} has played in is: {specificMMC[0]}")
    else:
        print(f"{NAME} has played in {len(specificMMC)} MMC's")
        print(f"The specific editions {NAME} has played in are:")
        num = 0
        for i in range(len(specificMMC) - 1):
            if num < 10:
                print(f"{specificMMC[i]}, ", end="")
                num += 1
            else:
                print(f"{specificMMC[i]}")
                num = 0
        print(specificMMC[-1])
    conn.close()
    return

In [3]:
import sqlite3
def winLossRecord(NAME):
    conn: sqlite3.Connection = sqlite3.connect("mmc.db")
    conn.execute("PRAGMA foreign_keys = 1")
    c: sqlite3.Cursor = conn.cursor()

    cnames = [] # for collecting all the challonge names a player has had
    cid = set() # for saving all of the challonge id's
    matchesWon = [] # for matches won 
    matchesLost = [] # for matches lost
    maps = {"maps won": 0, "maps lost": 0} # for tracking maps won and lost
    scores = {"1-0": 0, "0-1": 0, "2-0": 0, "0-2": 0, "2-1": 0, "1-2": 0, "3-0": 0, "3-1": 0, "3-2": 0, "0-3": 0, "1-3": 0, "2-3": 0}
    woWon = [] # for matches won by walkover
    woLost = [] # for matches lost by walkover

    # get the challonge names used on the brackets
    c.execute("SELECT * FROM ChallongeNames WHERE NAME = ?", (NAME,)) # query all challonge names a player has had
    results = c.fetchall()
    for r in results:
        cnames.append(r[0])

    # get the challonge id's that will appear in the matches
    for cn in cnames:
        c.execute("SELECT * FROM Participants WHERE CNAME = ?",(cn,))
        results = c.fetchall()
        for r in results:
            cid.add(r[0])

    # get all the matches a player has played
    for ids in cid:
        # get all the wins
        c.execute("SELECT * FROM MATCHES WHERE WINNERID = ?", (ids,))
        results = c.fetchall()
        for r in results:
            # check if it was a walkover
            if r[5] == -1:
                woWon.append(r)
            else:
                maps["maps won"] += r[4]
                maps["maps lost"] += r[5]
                scores[f"{r[4]}-{r[5]}"] += 1
                matchesWon.append(r)
        # get all thheh losses
        c.execute("SELECT * FROM MATCHES WHERE LOSERID = ?", (ids,))
        results = c.fetchall()
        for r in results:
            # check if it was a walkover
            if r[5] == -1:
                woLost.append(r)
            else:
                maps["maps won"] += r[5]
                maps["maps lost"] += r[4]
                scores[f"{r[5]}-{r[4]}"] += 1
                matchesLost.append(r)

    # display results
    print(f"{NAME} Career Record: {len(matchesWon)+len(woWon)}-{len(matchesLost)+len(woLost)}  Maps: {maps['maps won']}-{maps['maps lost']}")
    print(f"Wins: {len(matchesWon)}")
    print(f"Losses: {len(matchesLost)}")
    print(f"Wins by Walkover: {len(woWon)}")
    print(f"Losses by Walkover: {len(woLost)}")
    print("Map Scores:")
    num = 0
    for key, value in scores.items():
        if value > 0:
            if num < 5:
                print(f"{key}: {value}  ", end="")
                num += 1
            else:
                print(f"{key}: {value}  ")
                num = 0

    conn.close()

In [4]:
import sqlite3
def matchHistory(NAME1, NAME2):
    conn: sqlite3.Connection = sqlite3.connect("mmc.db")
    conn.execute("PRAGMA foreign_keys = 1")
    c: sqlite3.Cursor = conn.cursor()

    cnames1 = [] # for collecting all the challonge names NAME1 has had
    cnames2 = [] # for collecting all the challonge names NAME2 has had
    cid1 = set() # for saving all of the challonge id's for NAME1
    cid2 = set() # for saving all of the challonge id's for NAME2
    n1Wins = []
    n2Wins = []

    # get the challonge names used on the brackets for NAME1
    c.execute("SELECT * FROM ChallongeNames WHERE NAME = ?", (NAME1,)) # query all challonge names NAME1 has had
    results = c.fetchall()
    for r in results:
        cnames1.append(r[0])

    # get the challonge names used on the brackets for NAME2
    c.execute("SELECT * FROM ChallongeNames WHERE NAME = ?", (NAME2,)) # query all challonge names NAME2 has had
    results = c.fetchall()
    for r in results:
        cnames2.append(r[0])

    # get the challonge id's for NAME1
    for c1 in cnames1:
        c.execute("SELECT * FROM Participants WHERE CNAME = ?",(c1,))
        results = c.fetchall()
        for r in results:
            cid1.add(r[0])
    
    # get the challonge id's for NAME2
    for c2 in cnames2:
        c.execute("SELECT * FROM Participants WHERE CNAME = ?",(c2,))
        results = c.fetchall()
        for r in results:
            cid2.add(r[0])

    # check all combos of cids
    for c1 in cid1:
        for c2 in cid2:
            # wins for NAME1
            c.execute("SELECT * FROM Matches WHERE WINNERID = ? AND LOSERID = ?", (c1,c2))
            results = c.fetchall()
            for r in results:
                n1Wins.append(r)
            # wins for NAME2
            c.execute("SELECT * FROM Matches WHERE WINNERID = ? AND LOSERID = ?", (c2,c1))
            results = c.fetchall()
            for r in results:
                n2Wins.append(r)
    
    # if there are no matches between two players saw so
    if len(n1Wins) == 0 and len(n2Wins) == 0:
        print(f"{NAME1} and {NAME2} have never played each other in MMC")

    # Display Matches
    for match in n1Wins:
        c.execute("SELECT NUMBER,ELIMINATION,ROUNDS FROM MMC WHERE TOURNAMENTID = ?", (match[1],))
        mmc = c.fetchone()
        rnd = whatRound(match[6], mmc[1], mmc[2])
        print(f"{NAME1} {match[4]} - {match[5]} {NAME2}  MMC #{mmc[0]} {rnd}")
    for match in n2Wins:
        c.execute("SELECT NUMBER,ELIMINATION,ROUNDS FROM MMC WHERE TOURNAMENTID = ?", (match[1],))
        mmc = c.fetchone()
        rnd = whatRound(match[6], mmc[1], mmc[2])
        print(f"{NAME2} {match[4]} - {match[5]} {NAME1}  MMC #{mmc[0]} {rnd}")

In [25]:
import sqlite3
def generalRaceStatistics():
    conn: sqlite3.Connection = sqlite3.connect("mmc.db")
    conn.execute("PRAGMA foreign_keys = 1")
    c: sqlite3.Cursor = conn.cursor()

    # PvT Matches
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'p' AND LOSERRACE = 't'")
    results = c.fetchall()
    pvtWins = len(results)
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 't' AND LOSERRACE = 'p'")
    results = c.fetchall()
    pvtLosses = len(results)
    # PvZ Matches
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'p' AND LOSERRACE = 'z'")
    results = c.fetchall()
    pvzWins = len(results)
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'z' AND LOSERRACE = 'p'")
    results = c.fetchall()
    pvzLosses = len(results)
    # TvZ Matches
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 't' AND LOSERRACE = 'z'")
    results = c.fetchall()
    tvzWins = len(results)
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'z' AND LOSERRACE = 't'")
    results = c.fetchall()
    tvzLosses = len(results)
    # PvP Matches
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'p' AND LOSERRACE = 'p'")
    results = c.fetchall()
    pvpGames = len(results)
    # TvT Matches
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 't' AND LOSERRACE = 't'")
    results = c.fetchall()
    tvtGames = len(results)
    # ZvZ Matches
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'z' AND LOSERRACE = 'z'")
    results = c.fetchall()
    zvzGames = len(results)
    # RvP Matches
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'r' AND LOSERRACE = 'p'")
    results = c.fetchall()
    rvpWins = len(results)
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'p' AND LOSERRACE = 'r'")
    results = c.fetchall()
    rvpLosses = len(results)
    # RvT Matches
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'r' AND LOSERRACE = 't'")
    results = c.fetchall()
    rvtWins = len(results)
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 't' AND LOSERRACE = 'r'")
    results = c.fetchall()
    rvtLosses = len(results)
    # RvZ Matches
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'r' AND LOSERRACE = 'z'")
    results = c.fetchall()
    rvzWins = len(results)
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'z' AND LOSERRACE = 'r'")
    results = c.fetchall()
    rvzLosses = len(results)
    # RvR Matches
    c.execute("SELECT * FROM Matches WHERE WINNERRACE = 'r' AND LOSERRACE = 'r'")
    results = c.fetchall()
    rvrGames = len(results)


    print("Protoss Statistics:")
    print(f"PvZ Matches: {pvzLosses + pvzWins} | Match Record: {pvzWins} - {pvzLosses} | Win %: {round(pvzWins / (pvzWins + pvzLosses) * 100, 1)}")
    print(f"PvT Matches: {pvtLosses + pvtWins} | Match Record: {pvtWins} - {pvtLosses} | Win %: {round(pvtWins / (pvtWins + pvtLosses) * 100, 1)}")
    print(f"PvP Matches: {pvpGames}")
    print()
    print("Terran Statistics:")
    print(f"TvZ Matches: {tvzLosses + tvzWins} | Match Record: {tvzWins} - {tvzLosses} | Win %: {round(tvzWins / (tvzWins + tvzLosses) * 100, 1)}")
    print(f"TvP Matches: {pvtLosses + pvtWins} | Match Record: {pvtLosses} - {pvtWins} | Win %: {round(pvtLosses / (pvtWins + pvtLosses) * 100, 1)}")
    print(f"TvT Matches: {tvtGames}")
    print()
    print("Zerg Statistics:")
    print(f"ZvP Matches: {pvzLosses + pvzWins} | Match Record: {pvzLosses} - {pvzWins} | Win %: {round(pvzLosses / (pvzWins + pvzLosses) * 100, 1)}")
    print(f"ZvT Matches: {tvzLosses + tvzWins} | Match Record: {tvzLosses} - {tvzWins} | Win %: {round(tvzLosses / (tvzWins + tvzLosses) * 100, 1)}")
    print(f"ZvZ Matches: {zvzGames}")
    print()
    print("Random Statistics:")
    print(f"RvP Matches: {rvpWins + rvpLosses} | Match Record: {rvpWins} - {rvpLosses} | Win %: {round(rvpWins / (rvpWins + rvpLosses) * 100, 1)}")
    print(f"RvT Matches: {rvtWins + rvtLosses} | Match Record: {rvtWins} - {rvtLosses} | Win %: {round(rvtWins / (rvtWins + rvtLosses) * 100, 1)}")
    print(f"RvZ Matches: {rvzWins + rvzLosses} | Match Record: {rvzWins} - {rvzLosses} | Win %: {round(rvzWins / (rvzWins + rvzLosses) * 100, 1)}")
    print(f"RvR Matches: {rvrGames}")

    conn.close()
    return

### How to see how many MMC's a player has played in

In [20]:
# Who do you want to see the number of MMC's of?
NAME = "Kyboi"
numberOfMMC(NAME)


Kyboi has played in 36 MMC's
The specific editions Kyboi has played in are:
6, 9, 10, 30, 32, 33, 47, 48, 49, 50, 52
53, 54, 55, 77, 79, 81, 82, 83, 84, 85, 86
87, 88, 89, 90, 91, 92, 93, 96, 98, 100, 102
103, 104, 105


### How to see someones Win/Loss Record

In [19]:
# Who do you want to see the W/L record of?
NAME = "Kyboi"
winLossRecord(NAME)

Kyboi Career Record: 88-30  Maps: 191-82
Wins: 87
Losses: 28
Wins by Walkover: 1
Losses by Walkover: 2
Map Scores:
1-0: 8  0-1: 8  2-0: 42  0-2: 10  2-1: 23  1-2: 7  
3-0: 9  3-1: 2  3-2: 3  0-3: 1  2-3: 2  

### How to see two players Match History

In [7]:
# Who are the two players you want to see the match history of?
NAME1 = "Patches"
NAME2 = "CaptainClub"
matchHistory(NAME1,NAME2)

Patches 2 - 1 CaptainClub  MMC #97 Winners Final
Patches 2 - 0 CaptainClub  MMC #93 Winners Quarter Finals
CaptainClub 3 - 1 Patches  MMC #103 Finals
CaptainClub 2 - 0 Patches  MMC #63 Quarter Finals
CaptainClub 2 - 0 Patches  MMC #74 Ro16


In [28]:
# Testing Cell
import sqlite3
conn: sqlite3.Connection = sqlite3.connect("mmc.db")
conn.execute("PRAGMA foreign_keys = 1")
c: sqlite3.Cursor = conn.cursor()
generalRaceStatistics()

conn.close()

Protoss Statistics:
PvZ Matches: 394 | Match Record: 196 - 198 | Win %: 49.7
PvT Matches: 462 | Match Record: 219 - 243 | Win %: 47.4
PvP Matches: 257

Terran Statistics:
TvZ Matches: 344 | Match Record: 175 - 169 | Win %: 50.9
TvP Matches: 462 | Match Record: 243 - 219 | Win %: 52.6
TvT Matches: 192

Zerg Statistics:
ZvP Matches: 394 | Match Record: 198 - 196 | Win %: 50.3
ZvT Matches: 344 | Match Record: 169 - 175 | Win %: 49.1
ZvZ Matches: 143

Random Statistics:
RvP Matches: 10 | Match Record: 3 - 7 | Win %: 30.0
RvT Matches: 7 | Match Record: 3 - 4 | Win %: 42.9
RvZ Matches: 3 | Match Record: 0 - 3 | Win %: 0.0
RvR Matches: 0


### GUI for easy viewing

In [8]:
import PySimpleGUI as sg
import pandas as pd

csvFile = "Names.csv"
df = pd.read_csv(csvFile, encoding="utf-16", delimiter="\t")

# Extract the specific column (e.g., 'Name')
names = df["Normal Name"].drop_duplicates().tolist()

# Define the layout of your PySimpleGUI window
layout = [
    [sg.Text('List of Names:')],
    [sg.Input(key='-SEARCH-', enable_events=True, size=(20, 1)), sg.Button('Search')],
    [sg.Listbox(values=names, size=(30, 10), key='-LIST-', enable_events=True)],
    [sg.Column(layout=[[sg.Text('Enter name to see MMCs:')], [sg.InputText(key='-INPUT-')], [sg.Button('Show MMCs')],
                       [sg.Button('Win/Loss Record')]], vertical_alignment='top'),
     sg.Multiline(size=(40, 10), key='-OUTPUT-', disabled=True)],
    [sg.Text('Match History:', pad=((10, 0), (10, 0))), sg.InputText(key='-INPUT-NAME1-', size=(20, 1)),
     sg.Text('vs', pad=((10, 0), (10, 0))), sg.InputText(key='-INPUT-NAME2-', size=(20, 1)), sg.Button('Match History')],
    [sg.Multiline(size=(80, 10), key='-OUTPUT-MATCHHISTORY-', disabled=True)],
    [sg.Button('Exit')]
]

# Create the PySimpleGUI window
window = sg.Window('MMC Viewer', layout)

# Event loop
while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED or event == 'Exit':
        break
    elif event == '-LIST-':  # Handle listbox selection event
        selected_name = values['-LIST-'][0]  # Get the selected name from the listbox
        window['-INPUT-'].update(selected_name)  # Update the input field with the selected name
        output = numberOfMMC(selected_name)  # Calculate MMCs for the selected name
        window['-OUTPUT-'].update(output)  # Update the output field with the result
    elif event == 'Show MMCs':
        name = values['-INPUT-'].strip()
        if name:
            output = numberOfMMC(name)
            window['-OUTPUT-'].update(output)
    elif event == 'Search' or event == '-SEARCH-':
        search_term = values['-SEARCH-'].strip().lower()  # Get the search term and normalize to lowercase
        filtered_names = [name for name in names if search_term in name.lower()]
        window['-LIST-'].update(values=filtered_names)
    elif event == 'Win/Loss Record':
        name = values['-INPUT-'].strip()
        if name:
            output = winLossRecord(name)
            window['-OUTPUT-'].update(output)
    elif event == 'Match History':
        name1 = values['-INPUT-NAME1-'].strip()
        name2 = values['-INPUT-NAME2-'].strip()
        if name1 and name2:
            output = matchHistory(name1, name2)
            window['-OUTPUT-MATCHHISTORY-'].update(output)

window.close()

AttributeError: 'float' object has no attribute 'lower'

: 